In [1]:
import vrep
import time
import numpy as np
from scipy import linalg
import math
import random

# Forward Kinematics

In [2]:
def skew(m):
    return np.array([[0 ,-m[2], m[1]],[m[2], 0, -m[0]], [-m[1], m[0], 0]])


In [3]:
def revolute(a,q):
    ab = skew(a)
    bottom = -1*np.dot(ab,q)
    return np.array([[a[0]],[a[1]],[a[2]],[bottom[0]],[bottom[1]],[bottom[2]]])

In [4]:
def brack6(a):
    ws = a[:3]
    vs = a[3:]
    wb = skew(ws)
    ans = np.array([[wb[0,0], wb[0,1], wb[0,2], vs[0]],[wb[1,0], wb[1,1], wb[1,2], vs[1]],[wb[2,0], wb[2,1], wb[2,2], vs[2]], [0,0,0,0]])
    return ans

In [5]:
def adj(a):
    R = np.array(a[:3,:3])
    p = np.array(a[:3,3])
    ret_top = np.hstack((R, np.zeros((3,3))))
    ret_bottom = np.hstack((skew(p).dot(R), R))
    ret = np.vstack((ret_top,ret_bottom))
    return ret

In [6]:
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

In [7]:
def eulerAnglesToRotationMatrix(theta) :
     
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])
         
         
                     
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])
                 
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])
                     
                     
    R = np.dot(R_z, np.dot( R_y, R_x ))
 
    return R

In [8]:
def euclideanDistance(p1,p2):
    return math.sqrt(math.pow(p1[0]-p2[0],2)+math.pow(p1[1]-p2[1],2)+math.pow(p1[2]-p2[2],2))

In [9]:
def forward_kinematics(theta,screw,M=np.array([[0,0,-1, -0.1940], [0,1,0,0], [1,0,0,0.6511], [0,0,0,1]])):
    
    a0 = np.array([0,0,1])
    q0 = np.array([0,0,0.1045])
    
    a1 = np.array([-1,0,0])
    q1 = np.array([-0.115,0,0.1089])
    
    a2 = np.array([-1,0,0])
    q2 = np.array([-0.115,0,0.3525])
    
    a3 = np.array([-1,0,0]) #Not bad here
    q3 = np.array([-0.115,0,0.5658])
    
    a4 = np.array([0,0,1]) #this joint also doesn't look bad
    q4 = np.array([-0.112,0,0.65])
    
    a5 = np.array([-1,0,0])  #this doesn't look bad, but hard to tell with this joint
    q5 = np.array([-0.11,0,0.6511])
    
    
#     M = np.array([[0,0,-1, -0.1940], [0,1,0,0], [1,0,0,0.6511], [0,0,0,1]])
    
    axis = [a0,a1,a2,a3,a4,a5]
    point = [q0,q1,q2,q3,q4,q5]
    
#     screw = [revolute(axis[a],point[a]) for a in range(len(axis))]
    exp = [linalg.expm(brack6(screw[s])*theta[s]) for s in range (len(screw))]
    
    ex_multiplied = np.identity(4)
    for i in exp:
        ex_multiplied = ex_multiplied.dot(i)
    T = ex_multiplied.dot(M)
    return T

In [10]:
def finalpos(S, theta, start):
    """ This function finds the final position of all joints. Solution to 5.1.3.
    Parameters:
    S: the 6x6 matrix of the spatial screw axes for all joints.
    theta: a 6x1 matrix representing a certain configuration of the robot. 
    start: a 3x8 where the i'th column represents the initial position of the ith joint in terms of frame 0. """
    start = np.transpose(np.array(start))
    position = start[:,:2]
    S = np.array(S)
    for i in range(2,8):
        M = np.identity(4)
        M[0,3] = start[0, i]
        M[1,3] = start[1, i]
        M[2,3] = start[2, i]
        T = forward_kinematics(theta[0:i-1],np.transpose(S[:, 0:i-1])[0], M)
        position = np.concatenate((position, T[:3, 3:4]),axis=1)

    return position

In [11]:
def inverse_kinematics(T):
    a0 = np.array([0,0,1])
    q0 = np.array([0,0,0.1045])
    
    a1 = np.array([-1,0,0])
    q1 = np.array([-0.115,0,0.1089])
    
    a2 = np.array([-1,0,0])
    q2 = np.array([-0.115,0,0.3525])
    
    a3 = np.array([-1,0,0]) #Not bad here
    q3 = np.array([-0.115,0,0.5658])
    
    a4 = np.array([0,0,1]) #this joint also doesn't look bad
    q4 = np.array([-0.112,0,0.65])
    
    a5 = np.array([-1,0,0])  #this doesn't look bad, but hard to tell with this joint
    q5 = np.array([-0.11,0,0.6511])
    
    
    M = np.array([[0,0,-1, -0.1940], [0,1,0,0], [1,0,0,0.6511], [0,0,0,1]])
    
    axis = [a0,a1,a2,a3,a4,a5]
    point = [q0,q1,q2,q3,q4,q5]
    
    screw = [revolute(axis[a],point[a]) for a in range(len(axis))]
    
    theta = np.repeat(np.pi/8,6).reshape(6,1)
    tolerance = 0.01
    error = 20
    num_iterations = 0
    while (error > tolerance) and (num_iterations < 1000):
        exp = [linalg.expm(brack6(screw[s])*theta[s]) for s in range(len(screw))]
        Tc = np.identity(4)
        for i in exp:
            Tc = Tc.dot(i)
        Tc = Tc.dot(M)
        vb = linalg.logm(T.dot(np.linalg.inv(Tc)))
        v = np.array([vb[2,1],vb[0,2],vb[1,0],vb[0,3],vb[1,3],vb[2,3]]).reshape(6,1)
        
        jacobian = []
        jacobian.append(screw[0])
        j = np.identity(4)
        for i in range(len(exp)-1):
            j = j.dot(exp[i])
            jacobian.append(adj(j).dot(screw[i+1]))
            
        space = np.hstack((i for i in jacobian))
        
        thetadot = (np.linalg.inv(np.add(np.transpose(space).dot(space),0.1*np.eye(6)))).dot(np.transpose(space).dot(v))
        theta = np.add(theta,thetadot)
        n = np.linalg.norm(v)
        error = abs(n)
        num_iterations+=1
    print ("Iterations: ",num_iterations)
    if (num_iterations == 1000):
        return None
    return theta   

In [12]:
def isColliding(p1,p2,r):
    return euclideanDistance(p1,p2) < 2*r

In [13]:
def collision_detection(theta, obj, r, final_position):
    for i in range(len(final_position)):
        if isColliding(final_position[i],obj,r):
            return 1
        for j in range(i+1,len(final_position)-1):
            if isColliding(final_position[i],final_position[j],r):
                return 1
    return 0

In [14]:
def lin_collision_check(theta_start, theta_goal, screw, M, p_obstacle, r_obstacle):    
    base = np.array([0,0,0])
    q0 = np.array([0,0,0.1045])
    q1 = np.array([-0.115,0,0.1089])
    q2 = np.array([-0.115,0,0.3525])
    q3 = np.array([-0.115,0,0.5658])
    q4 = np.array([-0.112,0,0.65])
    q5 = np.array([-0.11,0,0.6511])
    end = np.array([-0.194,0,0.6511])
    theta_start = theta_start.reshape(6,1)
    theta_goal = theta_goal.reshape(6,1)
    for s in np.arange(0,1,0.01):
        theta = np.add(np.multiply(theta_goal,s),np.multiply(theta_start,(1-s)))
        T_end = forward_kinematics(theta,screw,M)
        start = [base, q0,q1,q2,q3,q4,q5, end]
        end = finalpos(screw,theta,start)
        for i in range(len(p_obstacle)):
            if collision_detection(theta,p_obstacle[i],r_obstacle[i], end):
                return 1
    return 0

In [15]:
def full_path(start_tree, end_tree, start_node, end_node, start_parent, end_parent):
    start_tree = np.transpose(np.array(start_tree))
    end_tree = np.transpose(np.array(end_tree))
    parent = end_parent[end_node]
    final_tree = end_tree[:,end_node:][0]
    while parent != 0:
#         theta = np.transpose(np.array([end_tree[0,parent],end_tree[1,parent],end_tree[2,parent]]))
        final_tree = np.hstack((final_tree,np.array(end_tree[:,parent:parent+1])))
        parent = end_parent[parent]
        
    parent = start_parent[start_node]
#     theta = np.transpose(np.array([start_tree[0,start_node],start_tree[1,start_node],start_tree[2,start_node]]))
    final_tree = np.hstack((np.array(start_tree[:,start_node:]),final_tree))
    while parent != 0:
#         theta = np.transpose(np.array([start_tree[0,parent],start_tree[1,parent],start_tree[2,parent]]))
        final_tree = np.hstack((np.array(start_tree[:,parent:parent+1]),final_tree))
        parent = start_parent[parent]
        
    return final_tree

In [16]:
def path_planning(p_obstacle, r_obstacle, theta_start, theta_goal):
    base = np.array([0,0,0])
    
    a0 = np.array([0,0,1])
    q0 = np.array([0,0,0.1045])
    
    a1 = np.array([-1,0,0])
    q1 = np.array([-0.115,0,0.1089])
    
    a2 = np.array([-1,0,0])
    q2 = np.array([-0.115,0,0.3525])
    
    a3 = np.array([-1,0,0]) #Not bad here
    q3 = np.array([-0.115,0,0.5658])
    
    a4 = np.array([0,0,1]) #this joint also doesn't look bad
    q4 = np.array([-0.112,0,0.65])
    
    a5 = np.array([-1,0,0])  #this doesn't look bad, but hard to tell with this joint
    q5 = np.array([-0.11,0,0.6511])
    
    end = np.array([-0.194,0,0.6511])
    
    M = np.array([[0,0,-1, -0.1940], [0,1,0,0], [1,0,0,0.6511], [0,0,0,1]])
    
    axis = [a0,a1,a2,a3,a4,a5]
    point = [q0,q1,q2,q3,q4,q5]
    start = [base, q0,q1,q2,q3,q4,q5, end]
    
    screw = [revolute(axis[a],point[a]) for a in range(len(axis))]
    
    num_obj = len(p_obstacle[0])
    robot_num = 8
    p_obstacle = np.vstack((p_obstacle,np.ones(num_obj)))
    num_joints = 6
    
    start_tree = []
    end_tree = []
    start_parent = []
    end_parent = []
    start_tree.append(theta_start)
    start_parent.append(0)
    end_tree.append(theta_goal)
    end_parent.append(0)
    
    if lin_collision_check(theta_start, theta_goal, screw, M, p_obstacle, r_obstacle):
        return full_path(start_tree, end_tree, 0 , 0, start_parent, end_parent)
    
    loop=True
    count = 0
    while loop:
        count += 1
        theta = np.random.uniform(low=-np.pi, high = np.pi, size = 6)
        final_position = finalpos(screw,theta,start)
        check = 0
        for i in range(len(p_obstacle)):
            if collision_detection(theta,p_obstacle[i],r_obstacle[i], final_position):
                check = 1
                break
        if (check!=1):
            min_start = 1000
            min_end = 1000
            for i in range(len(start_tree)):
                curr = np.linalg.norm(np.subtract(theta,start_tree[i])) 
                if curr < min_start:
                    min_start = curr
                    start_node = i
            for i in range(len(end_tree)):
                curr = np.linalg.norm(np.subtract(theta,end_tree[i])) 
                if curr < min_end:
                    min_end = curr
                    end_node = i
            if min_start < min_end:
                best_theta = start_tree(start_node)
            else:
                best_theta = end_tree(end_node)
                
            if (lin_collision_check(theta,best_theta,screw,M,p_obstacle,r_obstacle)) == 0:
                if min_start < min_end:
                    start_tree.append(theta)
                    start_parent.append(start_node)
                    if (lin_collision_check(theta,end_tree[end_node],screw,M,p_obstacle,r_obstacle)) == 0:
                        return full_path(start_tree, end_tree, start_tree.len()-1, end_node, start_parent, end_parent)
                else:
                    end_tree.append(theta)
                    end_parent.append(end_node)
                    if (lin_collision_check(theta,start_tree[start_node],screw,M,p_obstacle,r_obstacle)) == 0:
                        return full_path(start_tree, end_tree, start_node, end_tree.len()-1, start_parent, end_parent)
                    
            
        if (count == 1000):
            loop=False
        
    return list()

In [29]:
# Connect to V-REP (raise exception on failure)
clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
if clientID == -1:
    raise Exception('Failed connecting to remote API server')

# Start simulation
vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

time.sleep(2)
joint_handles = [0 for i in range(6)]
for i in range(6):
    result, joint_handles[i] = vrep.simxGetObjectHandle(clientID, 'UR3_joint'+str(i+1), vrep.simx_opmode_blocking)


result, connector = vrep.simxGetObjectHandle(clientID, 'BaxterVacuumCup', vrep.simx_opmode_blocking)
if result == -1:
    print ("Nah")

T = np.array([[0,1,0, 0.0], [0,0,1,0.29], [1,0,0,0.5866], [0,0,0,1]])
theta = inverse_kinematics(T)
if theta is None:
    print ("bleh")
else:
    vrep.simxPauseCommunication(clientID,True)
    for i in range(6):
        vrep.simxSetJointTargetPosition(clientID, joint_handles[i], theta[i], vrep.simx_opmode_oneshot)
    
    vrep.simxPauseCommunication(clientID,False)
time.sleep(1)
cup = "BaxterVacuumCup_active"
vrep.simxSetIntegerSignal(clientID, cup, 1, vrep.simx_opmode_oneshot)
time.sleep(1)


theta_home = [0 for i in range(6)]
if theta is None:
    print ("bleh")
else:
    vrep.simxPauseCommunication(clientID,True)
    for i in range(6):
        vrep.simxSetJointTargetPosition(clientID, joint_handles[i], theta_home[i], vrep.simx_opmode_oneshot)
    
    vrep.simxPauseCommunication(clientID,False)
time.sleep(1)

theta[0] = theta[0] + np.pi
if theta is None:
    print ("bleh")
else:
    vrep.simxPauseCommunication(clientID,True)
    for i in range(6):
        vrep.simxSetJointTargetPosition(clientID, joint_handles[i], theta[i], vrep.simx_opmode_oneshot)
    
    vrep.simxPauseCommunication(clientID,False)
time.sleep(1) 
#pour out cup
theta[5] = theta[5] + np.pi
if theta is None:
    print ("bleh")
else:
    vrep.simxPauseCommunication(clientID,True)
    for i in range(6):
        vrep.simxSetJointTargetPosition(clientID, joint_handles[i], theta[i], vrep.simx_opmode_oneshot)
    
    vrep.simxPauseCommunication(clientID,False) 

#result, fire = vrep.simxGetObjectHandle(clientID, "fire", vrep.simx_opmode_blocking)
#vrep.simxRemoveObject(clientID, fire, vrep.simx_opmode_blocking)

#Put Cup Back on Shelf
time.sleep(1)
theta[0] = theta[0] - np.pi/2
theta[5] = theta[5] + np.pi
if theta is None:
    print ("bleh")
else:
    vrep.simxPauseCommunication(clientID,True)
    for i in range(6):
        vrep.simxSetJointTargetPosition(clientID, joint_handles[i], theta[i], vrep.simx_opmode_oneshot)
    
    vrep.simxPauseCommunication(clientID,False) 
#Let Go of Cup
time.sleep(1)
vrep.simxSetIntegerSignal(clientID, cup, 0, vrep.simx_opmode_oneshot)

time.sleep(1)

#return to home position
vrep.simxPauseCommunication(clientID,True)
for i in range(6):
    vrep.simxSetJointTargetPosition(clientID, joint_handles[i], theta_home[i], vrep.simx_opmode_oneshot)
vrep.simxPauseCommunication(clientID,False)


time.sleep(5)
# Stop simulation
vrep.simxStopSimulation(clientID, vrep.simx_opmode_oneshot)

# Before closing the connection to V-REP, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
vrep.simxGetPingTime(clientID)

# Close the connection to V-REP
vrep.simxFinish(clientID)


Iterations:  214
